# 下載kaggle資料集

In [ ]:
api_token = {"username":"yanghsinyu","key":"812b9f8db55e347f14ae82e38fb9f2e9"}

import json
import zipfile
import os

if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

if not os.path.exists("/kaggle"):
    os.makedirs("/kaggle")
os.chdir('/kaggle')
!kaggle datasets download -d andrewmvd/face-mask-detection

In [ ]:
!ls /kaggle
!unzip "/kaggle/face-mask-detection"

In [ ]:
!rm -rf '/kaggle/annotations'
!rm -rf '/kaggle/face-mask-detection.zip'

# GAN

In [ ]:
import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image
import torchvision.datasets as dset

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
#model儲存路徑
dest = '/content/gan/'
#資料集路徑
base_dir ='/kaggle/'
os.makedirs(dest, exist_ok=True)

#參數設定
parser = argparse.ArgumentParser()
parser.add_argument('-f')
parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=12, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.002, help="learning rate")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=64, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=3, help="number of image channels")
parser.add_argument("--n_critic", type=int, default=5, help="number of training steps for discriminator per iter")
parser.add_argument("--clip_value", type=float, default=0.01, help="lower and upper clip value for disc. weights")
parser.add_argument("--sample_interval", type=int, default=400, help="interval betwen image samples")
opt = parser.parse_args()
print(opt)

img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False
if(cuda):
  print('TURE')
else:
  print('FALSE')

# Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
                #(100)*1*1
                nn.ConvTranspose2d( opt.latent_dim, 256, 3, 1, 0, bias=False),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.2, inplace=True),

                #(256)*((1-1)*1-2*0+3=3)*3
                nn.ConvTranspose2d( 256, 128, 3, 2, 0, bias=False),
                nn.BatchNorm2d(128),
                nn.LeakyReLU(0.2, inplace=True),

                #(128)*((3-1)*2-2*0+3=7)*7
                nn.ConvTranspose2d( 128, 64, 3, 2, 0, bias=False),
                nn.BatchNorm2d(64),
                nn.LeakyReLU(0.2, inplace=True),

                #(64)*((7-1)*2-2*0+3=15)*15
                nn.ConvTranspose2d( 64, 32, 3, 2, 0, bias=False),
                nn.BatchNorm2d(32),
                nn.LeakyReLU(0.2, inplace=True),

                #(32)*((15-1)*2-2*0+3=31)*31
                nn.ConvTranspose2d( 32, opt.channels, 4, 2, 0, bias=False),
                nn.BatchNorm2d(opt.channels),

                #(3)*((31-1)*2-2*0+4=64)*64
                nn.Tanh()
                )



    def forward(self, z):

        z = z.view(opt.batch_size, opt.latent_dim, 1, 1)
        img = self.model(z)

        return img

# Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.input_dim = opt.channels
        self.output_dim = 1
        self.input_size = int(np.prod(img_shape))

        self.model = nn.Sequential(
            # input is (3) x 64 x 64
            nn.Conv2d(opt.channels, 64, 5, 1, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # (64) x ((64-5+2*1)/1+1=62)*62
            nn.MaxPool2d(kernel_size = 2),

            # state size. (64) x 31 x 31
            nn.Conv2d(64, 128, 3, 1, 2, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            # (128) x ((31-3+2*1)/2+1=16) x 16
            nn.MaxPool2d(kernel_size = 2),

            # state size. (128) x 8 x 8
            nn.Conv2d(128, 256, 3, 1, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            # (256) x ((8-3+2*1)/1+1=8) x 8
            nn.MaxPool2d(kernel_size = 2),

            # state size. (256) x 4 x 4
            nn.Conv2d(256, 1, 4, 1, 0, bias=False),
        )

    def forward(self, img):
        validity  = self.model(img)
        return validity

# 使用GPU

In [ ]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()

#讀取資料集

In [ ]:
dataloader = torch.utils.data.DataLoader(
    dataset = dset.ImageFolder(root=base_dir, transform=transforms.Compose([
                               transforms.Resize(opt.img_size),
                               transforms.CenterCrop(opt.img_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])),
    batch_size=opt.batch_size,
    shuffle=True,
)
print(dataloader.dataset)

#

In [ ]:
# Optimizers
optimizer_G = torch.optim.RMSprop(generator.parameters(), lr=opt.lr)
optimizer_D = torch.optim.RMSprop(discriminator.parameters(), lr=opt.lr)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
os.makedirs(dest+"model/checkpoint/", exist_ok=True)
# 定義要保存檢查點的路徑和檔名
checkpoint_path = dest+'model/checkpoint/'

# Train

In [ ]:
# ----------
#  Training
# ----------
batches_done = 0
for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):


        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z).detach()
        # Adversarial loss
        loss_D = -torch.mean(discriminator(real_imgs)) + torch.mean(discriminator(fake_imgs))

        loss_D.backward()
        optimizer_D.step()

        # Clip weights of discriminator
        for p in discriminator.parameters():
            p.data.clamp_(-opt.clip_value, opt.clip_value)

        # Train the generator every n_critic iterations
        if i % opt.n_critic == 0:

          # -----------------
          #  Train Generator
          # -----------------

          optimizer_G.zero_grad()

          # Generate a batch of images
          gen_imgs = generator(z)
          # Adversarial loss
          loss_G = -torch.mean(discriminator(gen_imgs))

          loss_G.backward()
          optimizer_G.step()

          if i % 100 == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch+1, opt.n_epochs, batches_done % len(dataloader), len(dataloader), loss_D.item(), loss_G.item())
            )

        batches_done += 1

    if (epoch+1) % 20 == 0:
    	  # 建立一個字典來保存模型、優化器和其他相關資訊
        checkpoint = {
     	        'epoch': epoch,
      	        'model_G_state_dict': generator.state_dict(),
      	        'model_D_state_dict': discriminator.state_dict(),
       	        # 如果需要保存其他相關資訊，可以在此添加
        }

    	  # 使用`torch.save`函式保存檢查點到指定的路徑
        torch.save(checkpoint,checkpoint_path+"checkpoint_%d.pth" %(epoch+1))
    save_image(gen_imgs.data[:25], dest+"%d-%d.png" % (epoch+1, batches_done % len(dataloader)), nrow=5, normalize=True)

# 產生亂數z

In [ ]:
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt

# Sample noise as generator input
z = Variable(Tensor(np.random.normal(0, 1, (opt.batch_size, opt.latent_dim))))
z1 = Variable(Tensor(np.random.normal(0, 1, (opt.batch_size, opt.latent_dim))))
z2 = Variable(Tensor(np.random.normal(0, 1, (opt.batch_size, opt.latent_dim))))
z3 = Variable(Tensor(np.random.normal(0, 1, (opt.batch_size, opt.latent_dim))))
z4 = Variable(Tensor(np.random.normal(0, 1, (opt.batch_size, opt.latent_dim))))

print(z.size())

# 生成成圖片 ( batch_size張, 3x64x64 )

In [ ]:
#zn = Variable(Tensor(np.random.normal(0, 1, (opt.img_size, opt.latent_dim))))
# Assuming generator is your PyTorch generator model
fake_imgs = generator(z)
fake_imgs1 = generator(z1)
fake_imgs2 = generator(z2)
fake_imgs3 = generator(z3)
fake_imgs4 = generator(z4)
print("fake_imgs\n")
print(fake_imgs.min())

# Convert generated images to numpy arrays
fake_imgs_np = fake_imgs.cpu().detach().numpy()
fake_imgs_np1 = fake_imgs1.cpu().detach().numpy()
fake_imgs_np2 = fake_imgs2.cpu().detach().numpy()
fake_imgs_np3 = fake_imgs3.cpu().detach().numpy()
fake_imgs_np4 = fake_imgs4.cpu().detach().numpy()

# Rearrange the dimensions for color images (assuming RGB)
fake_imgs_np = np.transpose(fake_imgs_np* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)
fake_imgs_np1 = np.transpose(fake_imgs_np1* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)
fake_imgs_np2 = np.transpose(fake_imgs_np2* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)
fake_imgs_np3 = np.transpose(fake_imgs_np3* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)
fake_imgs_np4 = np.transpose(fake_imgs_np4* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)


# Define a function to display multiple images
def show_images(images, titles):
    num_images = len(images)
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(images[i])
        plt.title(titles[i])
        plt.axis('off')
    plt.show()

# Display the generated images using matplotlib
show_images([fake_imgs_np[0], fake_imgs_np1[0], fake_imgs_np2[0], fake_imgs_np3[0], fake_imgs_np4[0]], ['Image1', 'Image2', 'Image3', 'Image4', 'Image5'])

#lode model

In [ ]:
###讀取訓練好的模型
checkpoint = torch.load('/content/gan/model/checkpoint/checkpoint_200.pth')
generator.load_state_dict(checkpoint['model_G_state_dict'])
discriminator.load_state_dict(checkpoint['model_D_state_dict'])
start_epoch = checkpoint['epoch']

# 生成成圖片 ( batch_size張, 3x64x64 )

In [ ]:
#zn = Variable(Tensor(np.random.normal(0, 1, (opt.img_size, opt.latent_dim))))
# Assuming generator is your PyTorch generator model
fake_imgs = generator(z)
fake_imgs1 = generator(z1)
fake_imgs2 = generator(z2)
fake_imgs3 = generator(z3)
fake_imgs4 = generator(z4)
print("fake_imgs\n")
print(fake_imgs.min())

# Convert generated images to numpy arrays
fake_imgs_np = fake_imgs.cpu().detach().numpy()
fake_imgs_np1 = fake_imgs1.cpu().detach().numpy()
fake_imgs_np2 = fake_imgs2.cpu().detach().numpy()
fake_imgs_np3 = fake_imgs3.cpu().detach().numpy()
fake_imgs_np4 = fake_imgs4.cpu().detach().numpy()

# Rearrange the dimensions for color images (assuming RGB)
fake_imgs_np = np.transpose(fake_imgs_np* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)
fake_imgs_np1 = np.transpose(fake_imgs_np1* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)
fake_imgs_np2 = np.transpose(fake_imgs_np2* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)
fake_imgs_np3 = np.transpose(fake_imgs_np3* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)
fake_imgs_np4 = np.transpose(fake_imgs_np4* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)


# Define a function to display multiple images
def show_images(images, titles):
    num_images = len(images)
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(images[i])
        plt.title(titles[i])
        plt.axis('off')
    plt.show()

# Display the generated images using matplotlib
show_images([fake_imgs_np[0], fake_imgs_np1[0], fake_imgs_np2[0], fake_imgs_np3[0], fake_imgs_np4[0]], ['Image1', 'Image2', 'Image3', 'Image4', 'Image5'])

#訓練

In [ ]:
# ----------
#  Training
# ----------

#要訓練的epoch數量
end_epoch = 5

batches_done = 0
for epoch in range(start_epoch, start_epoch+end_epoch):
    for i, (imgs, _) in enumerate(dataloader):


        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z).detach()
        # Adversarial loss
        loss_D = -torch.mean(discriminator(real_imgs)) + torch.mean(discriminator(fake_imgs))

        loss_D.backward()
        optimizer_D.step()

        # Clip weights of discriminator
        for p in discriminator.parameters():
            p.data.clamp_(-opt.clip_value, opt.clip_value)

        # Train the generator every n_critic iterations
        if i % opt.n_critic == 0:

          # -----------------
          #  Train Generator
          # -----------------

          optimizer_G.zero_grad()

          # Generate a batch of images
          gen_imgs = generator(z)
          # Adversarial loss
          loss_G = -torch.mean(discriminator(gen_imgs))

          loss_G.backward()
          optimizer_G.step()

          if i % 100 == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch+1, opt.n_epochs, batches_done % len(dataloader), len(dataloader), loss_D.item(), loss_G.item())
            )

        batches_done += 1

    if (epoch+1) % 20 == 0:
    	  # 建立一個字典來保存模型、優化器和其他相關資訊
        checkpoint = {
     	        'epoch': epoch,
      	        'model_G_state_dict': generator.state_dict(),
      	        'model_D_state_dict': discriminator.state_dict(),
       	        # 如果需要保存其他相關資訊，可以在此添加
        }

    	  # 使用`torch.save`函式保存檢查點到指定的路徑
        torch.save(checkpoint,checkpoint_path+"checkpoint_%d.pth" %(epoch+1))
    save_image(gen_imgs.data[:25], dest+"%d-%d.png" % (epoch+1, batches_done % len(dataloader)), nrow=5, normalize=True)

# 生成成圖片 ( batch_size張, 3x64x64 )

In [ ]:
#zn = Variable(Tensor(np.random.normal(0, 1, (opt.img_size, opt.latent_dim))))
# Assuming generator is your PyTorch generator model
fake_imgs = generator(z)
fake_imgs1 = generator(z1)
fake_imgs2 = generator(z2)
fake_imgs3 = generator(z3)
fake_imgs4 = generator(z4)
print("fake_imgs\n")
print(fake_imgs.min())

# Convert generated images to numpy arrays
fake_imgs_np = fake_imgs.cpu().detach().numpy()
fake_imgs_np1 = fake_imgs1.cpu().detach().numpy()
fake_imgs_np2 = fake_imgs2.cpu().detach().numpy()
fake_imgs_np3 = fake_imgs3.cpu().detach().numpy()
fake_imgs_np4 = fake_imgs4.cpu().detach().numpy()

# Rearrange the dimensions for color images (assuming RGB)
fake_imgs_np = np.transpose(fake_imgs_np* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)
fake_imgs_np1 = np.transpose(fake_imgs_np1* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)
fake_imgs_np2 = np.transpose(fake_imgs_np2* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)
fake_imgs_np3 = np.transpose(fake_imgs_np3* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)
fake_imgs_np4 = np.transpose(fake_imgs_np4* 0.5 + 0.5, (0, 2, 3, 1))  # from (batch_size, 3, 64, 64) to (batch_size, 64, 64, 3)


# Define a function to display multiple images
def show_images(images, titles):
    num_images = len(images)
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(images[i])
        plt.title(titles[i])
        plt.axis('off')
    plt.show()

# Display the generated images using matplotlib
show_images([fake_imgs_np[0], fake_imgs_np1[0], fake_imgs_np2[0], fake_imgs_np3[0], fake_imgs_np4[0]], ['Image1', 'Image2', 'Image3', 'Image4', 'Image5'])

# save model

In [ ]:
# 建立一個字典來保存模型、優化器和其他相關資訊
checkpoint = {
     	  'epoch': epoch,
      	'model_G_state_dict': generator.state_dict(),
      	'model_D_state_dict': discriminator.state_dict(),
       	# 如果需要保存其他相關資訊，可以在此添加
}

# 使用`torch.save`函式保存檢查點到指定的路徑
torch.save(checkpoint,checkpoint_path+"checkpoint_%d.pth" %(epoch+1))